In [5]:
import wrds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cvxpy as cp
from arch import arch_model

# Connect to WRDS
db = wrds.Connection(wrds_username='simengut')


Loading library list...
Done


In [6]:
#tickers = ['ICLN', 'TAN', 'PBW', 'XLE', 'XOP', 'VDE']
tickers = ['QCLN', 'ACES', 'IXC', 'FENY', 'ICLN', 'TAN', 'PBW', 'XLE', 'XOP', 'VDE']


# Retrieve permnos from WRDS (CRSP)
permnos_query = f"""
    SELECT DISTINCT ON (ticker) permno, ticker 
    FROM crsp.stocknames
    WHERE ticker IN ({','.join(f"'{ticker}'" for ticker in tickers)})
    ORDER BY ticker, nameenddt DESC
    """
permnos_df = db.raw_sql(permnos_query)

# Create a mapping of tickers to their most recent permnos
ticker_to_permno = dict(zip(permnos_df['ticker'], permnos_df['permno']))
permnos = list(ticker_to_permno.values())

permnos

[17912, 14201, 92720, 89235, 90618, 91827, 17837, 90347, 86454, 91318]

In [7]:
ticker_to_permno

{'ACES': 17912,
 'FENY': 14201,
 'ICLN': 92720,
 'IXC': 89235,
 'PBW': 90618,
 'QCLN': 91827,
 'TAN': 17837,
 'VDE': 90347,
 'XLE': 86454,
 'XOP': 91318}

### Find which data has missing values and not


In [8]:
# Initialize dictionary to store DataFrames for each ticker
ticker_dfs = {}

# Retrieve data for each ticker separately
for ticker, permno in ticker_to_permno.items():
    print(f"\nRetrieving data for {ticker} (permno: {permno})")
    
    query = f"""
        SELECT date, permno, ret 
        FROM crsp.msf
        WHERE permno = {permno}
        AND date BETWEEN '2007-01-01' AND '2024-12-31'
        AND ret IS NOT NULL
        ORDER BY date
        """
    try:
        df = db.raw_sql(query, date_cols=['date'])
        if len(df) > 0:
            print(f"Retrieved {len(df)} rows")
            print(f"Date range: {df['date'].min()} to {df['date'].max()}")
            print(f"Missing values: {df['ret'].isnull().sum()}")
            ticker_dfs[ticker] = df
        else:
            print(f"No data found for {ticker}")
        print("-" * 50)
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {str(e)}")
        print("-" * 50)

# Display summary for all tickers
print("\nSummary for all tickers:")
for ticker, df in ticker_dfs.items():
    print(f"\n{ticker}:")
    print(f"Date range: {df['date'].min()} to {df['date'].max()}")
    print(f"Number of observations: {len(df)}")
    print(f"Missing values: {df['ret'].isnull().sum()}")


Retrieving data for ACES (permno: 17912)
Retrieved 78 rows
Date range: 2018-07-31 00:00:00 to 2024-12-31 00:00:00
Missing values: 0
--------------------------------------------------

Retrieving data for FENY (permno: 14201)
Retrieved 134 rows
Date range: 2013-11-29 00:00:00 to 2024-12-31 00:00:00
Missing values: 0
--------------------------------------------------

Retrieving data for ICLN (permno: 92720)
Retrieved 198 rows
Date range: 2008-07-31 00:00:00 to 2024-12-31 00:00:00
Missing values: 0
--------------------------------------------------

Retrieving data for IXC (permno: 89235)
Retrieved 216 rows
Date range: 2007-01-31 00:00:00 to 2024-12-31 00:00:00
Missing values: 0
--------------------------------------------------

Retrieving data for PBW (permno: 90618)
Retrieved 216 rows
Date range: 2007-01-31 00:00:00 to 2024-12-31 00:00:00
Missing values: 0
--------------------------------------------------

Retrieving data for QCLN (permno: 91827)
Retrieved 214 rows
Date range: 2007-

### Here is actual data to use!! 

In [4]:
tickers = ['ICLN', 'PBW', 'QCLN', 'IXC', 'VDE', 'XLE']
permnos = [permno for ticker, permno in ticker_to_permno.items() if ticker in tickers]

# Initialize empty dataframe
final_df = pd.DataFrame()

# Retrieve data for each ticker separately and merge
for ticker in tickers:
    permno = ticker_to_permno[ticker]
    query = f"""
        SELECT date, ret 
        FROM crsp.msf
        WHERE permno = {permno}
        AND date BETWEEN '2008-07-01' AND '2024-12-31'
        ORDER BY date
    """
    df = db.raw_sql(query, date_cols=['date'])
    df.rename(columns={'ret': ticker}, inplace=True)
    
    if final_df.empty:
        final_df = df
    else:
        final_df = pd.merge(final_df, df, on='date', how='inner')

# Check for missing values explicitly
print(final_df.isnull().sum())
print(f"Final DataFrame shape: {final_df.shape}")

returns_pivot = final_df.copy()



date    0
ICLN    0
PBW     0
QCLN    0
IXC     0
VDE     0
XLE     0
dtype: int64
Final DataFrame shape: (198, 7)


In [9]:
returns_pivot

# make date teh index
returns_pivot.set_index('date', inplace=True)
returns_pivot


,ICLN,PBW,QCLN,IXC,VDE,XLE
date,,,,,,
2008-07-31,-0.02607,-0.026087,-0.051933,-0.137549,-0.144426,-0.159132
2008-08-29,0.024518,0.039391,0.06698,-0.009118,-0.006781,0.00336
2008-09-30,-0.272376,-0.249116,-0.242303,-0.132505,-0.151297,-0.149162
2008-10-31,-0.397254,-0.335128,-0.335092,-0.178202,-0.200472,-0.187994
2008-11-28,-0.12824,-0.140688,-0.173135,-0.017748,-0.028173,-0.02179
...,...,...,...,...,...,...
2024-08-30,0.009894,-0.102505,-0.036865,-0.004509,-0.024795,-0.020706
2024-09-30,0.027992,0.038776,0.02036,-0.035042,-0.030666,-0.030155
2024-10-31,-0.108237,0.000998,-0.076859,-0.000741,0.009553,0.008998
